In [37]:
import mne_bids
from glob import glob
import mne
import re
import numpy as np
import pandas as pd
from collections import defaultdict
from mne.parallel import parallel_func

from wrangler import Wrangler
from classifier import Classifier
import matplotlib.pyplot as plt
import tqdm as tqdm
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
DATA_DIR = r"E:\datasets\supracapacity\derivatives"
EXPERIMENT_NAME='supracapacity'

dropped_chans={
    'eeg':['Fp1','Fp2'],
    'eog':'ALL',
    'eyegaze':'ALL',
    'pupil':'ALL',
    'misc':'ALL'
}

conditions = ['memory/ss2/no_placeholders','memory/ss2/placeholders',
                'memory/ss4/no_placeholders','memory/ss4/placeholders',
                'memory/ss6/no_placeholders','memory/ss6/placeholders',
                'memory/ss8/no_placeholders','memory/ss8/placeholders',
                'attention/ss2/no_placeholders','attention/ss2/placeholders',
                'attention/ss4/no_placeholders','attention/ss4/placeholders',
                'attention/ss6/no_placeholders','attention/ss6/placeholders',
                'attention/ss8/no_placeholders','attention/ss8/placeholders']

training_groups = [('memory','ss2'),('memory','ss4')]
testing_groups =  [('memory','ss2'),('memory','ss4')]

group_dict = defaultdict(None)
for igroup,group in enumerate(training_groups):
    for condition in conditions:
        if all([subgroup in condition.split('/') for subgroup in group]):
            group_dict[condition] = igroup
for igroup,group in enumerate(testing_groups):
    if group not in training_groups:
        for condition in conditions:
            if all([subgroup in condition.split('/') for subgroup in group]):
                    group_dict[condition] = 2+igroup



In [42]:
wrangl = Wrangler(data_dir=DATA_DIR,experiment_name=EXPERIMENT_NAME,dropped_chans=dropped_chans,conditions=conditions,training_groups=training_groups,testing_groups=testing_groups,n_folds=1000)
clfr = Classifier()

In [43]:

accs = []
accs_shuff = []
confusion_mats = []
confidence_scores = []
for isub in tqdm.tqdm(range(wrangl.nsub)):
    try:
        Xdata,ydata = wrangl.load_eeg(isub)
    except:
        continue
    parallel,p_func,_ = parallel_func(clfr.decode_across_time,n_jobs = -1)

    out = parallel(
        p_func(
            X_train=X_train,
            X_test=X_test,
            y_train=y_train,
            y_test=y_test,
        )
        for X_train,X_test,y_train,y_test in wrangl.bin_and_split(Xdata,ydata)
    )
    acc,acc_shuff,conf_mat,score = map(np.stack,zip(*out))
    accs.append(acc)
    accs_shuff.append(acc_shuff)
    confusion_mats.append(conf_mat)
    confidence_scores.append(score)

accs = np.stack(accs)
accs_shuff = np.stack(accs_shuff)
confusion_mats = np.stack(confusion_mats)
confidence_scores = np.stack(confidence_scores)

  3%|▎         | 1/31 [00:44<22:10, 44.36s/it]


FileNotFoundError: [Errno 2] No such file or directory: 'E:\\datasets\\supracapacity\\derivatives\\sub-03\\eeg\\sub-03_task-supracapacity_desc-preprocessed_rejection_flags.npy'